In [1]:
import numpy as np
import pandas as pd
import QUANTAXIS as qa
import datetime as dt

QUANTAXIS>> start QUANTAXIS


no display found. Using non-interactive Agg backend
if you use ssh, you can use ssh with -X parmas to avoid this issue


QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.0.68
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##`````##```##```````##```````##````##```````###````````##``````###```` 
  ##````````##```##````````##`````##````````##````##``````##``##`````

In [2]:
today = dt.datetime.today()
start_date = today - dt.timedelta(days=365)
stocks = qa.QA_fetch_stock_list_adv().code.tolist()

In [35]:
cands = qa.QA_fetch_stock_day_adv(stocks, start=start_date.strftime('%Y-%m-%d'), end=today.strftime('%Y-%m-%d'))


In [34]:
def lift_stop(data):
    data['lift'] = data.close / data.close.shift(1) - 1
    data['stop'] = (data.lift > 0.095) * 1
    data['uplift'] = ((data.lift > 0.07) & (data.lift <= 0.095)) * 1
    return data.loc[:, ['uplift', 'stop']]

In [36]:
data = cands.add_func(lift_stop)

In [37]:
data2 = data.loc[:, ['uplift', 'stop']].reset_index()
data2.head()

,date,code,uplift,stop
0,2017-07-31,000001,0,0
1,2017-07-31,000002,0,0
2,2017-07-31,000004,0,0
3,2017-07-31,000005,0,0
4,2017-07-31,000006,0,0


In [38]:
data2 = data2.sort_values(['code', 'date'], ascending=[True, False])


In [39]:
data2['cumstop']= data2.groupby('code')['stop'].transform(lambda x: x.cumsum())

In [51]:
data2.loc[data2.code=='000063', :].head(10)

,date,code,uplift,stop,cumstop
788777,2018-07-30,000063,0,0,0
785394,2018-07-27,000063,0,0,0
782011,2018-07-26,000063,0,0,0
778625,2018-07-25,000063,0,0,0
775242,2018-07-24,000063,0,0,0
771858,2018-07-23,000063,0,0,0
768475,2018-07-20,000063,0,0,0
765096,2018-07-19,000063,0,0,0
761719,2018-07-18,000063,0,0,0
758346,2018-07-17,000063,0,1,1


In [24]:
data.head(10)

,date,code,lift,stop,cumstop
788731,2018-07-30,000001,0,0,0
785348,2018-07-27,000001,0,0,0
781965,2018-07-26,000001,0,0,0
778579,2018-07-25,000001,0,0,0
775196,2018-07-24,000001,0,0,0
771812,2018-07-23,000001,0,0,0
768429,2018-07-20,000001,0,0,0
765050,2018-07-19,000001,0,0,0
761673,2018-07-18,000001,0,0,0
758300,2018-07-17,000001,0,0,0


In [25]:
data.shift(1).head(10)

,date,code,lift,stop,cumstop
788731,NaT,NaN,NaN,NaN,NaN
785348,2018-07-30,000001,0.0,0.0,0.0
781965,2018-07-27,000001,0.0,0.0,0.0
778579,2018-07-26,000001,0.0,0.0,0.0
775196,2018-07-25,000001,0.0,0.0,0.0
771812,2018-07-24,000001,0.0,0.0,0.0
768429,2018-07-23,000001,0.0,0.0,0.0
765050,2018-07-20,000001,0.0,0.0,0.0
761673,2018-07-19,000001,0.0,0.0,0.0
758300,2018-07-18,000001,0.0,0.0,0.0


In [41]:
data = cands.add_func(lift_stop)
data = data.loc[:, ['uplift', 'stop']].reset_index()
data = data.sort_values(['code', 'date'], ascending=[True, False])
data['cumstop']= data.groupby('code')['stop'].transform(lambda x: x.cumsum())
data['cumlift']= data.groupby('code')['uplift'].transform(lambda x: x.cumsum())
data['intensity'] = data.cumstop + data.cumlift * 0.5


In [46]:
data['interval'] = (today - data.date).apply(lambda x: x.days)
data.head(10)

,date,code,uplift,stop,cumstop,cumlift,intensity,interval
788731,2018-07-30,000001,0,0,0,0,0.0,1
785348,2018-07-27,000001,0,0,0,0,0.0,4
781965,2018-07-26,000001,0,0,0,0,0.0,5
778579,2018-07-25,000001,0,0,0,0,0.0,6
775196,2018-07-24,000001,0,0,0,0,0.0,7
771812,2018-07-23,000001,0,0,0,0,0.0,8
768429,2018-07-20,000001,0,0,0,0,0.0,11
765050,2018-07-19,000001,0,0,0,0,0.0,12
761673,2018-07-18,000001,0,0,0,0,0.0,13
758300,2018-07-17,000001,0,0,0,0,0.0,14


In [47]:
data['rank'] = data.groupby('interval')['intensity'].transform(lambda x: x.rank(ascending=False))

In [50]:
data.loc[data['rank'] < 10, :].head(10)

,date,code,uplift,stop,cumstop,cumlift,intensity,interval,rank
785360,2018-07-27,000017,0,1,2,0,2.0,4,5.0
781977,2018-07-26,000017,0,1,3,0,3.0,5,2.5
778591,2018-07-25,000017,0,1,4,0,4.0,6,2.5
775208,2018-07-24,000017,0,0,4,0,4.0,7,6.5
785451,2018-07-27,000509,0,1,2,0,2.0,4,5.0
782068,2018-07-26,000509,0,1,3,0,3.0,5,2.5
778682,2018-07-25,000509,0,1,4,0,4.0,6,2.5
775299,2018-07-24,000509,0,0,4,0,4.0,7,6.5
785487,2018-07-27,000557,0,1,2,0,2.0,4,5.0
514790,2018-03-28,000613,0,1,10,3,11.5,125,9.0


In [ ]:
cans = qa.QA_fetch_stock_day_adv('', start=start_date.strftime('%Y-%m-%d'), end=today.strftime('%Y-%m-%d'))
